# 0. 환경설정

In [53]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# 시각화 관련
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from collections import Counter
from PIL import Image
from matplotlib import rc

# 그래프에서 한글 폰트 깨지는 문제에 대한 대처(전역 글꼴 설정)
import matplotlib.font_manager as fm
import warnings
warnings.filterwarnings(action='ignore') 

import matplotlib
matplotlib.rcParams['font.family'] ='Malgun Gothic'
matplotlib.rcParams['axes.unicode_minus'] =False

In [54]:
ex = pd.read_csv('서울특별시_보행자우선도로_전체.csv', encoding='cp949')
ex.head()

FileNotFoundError: [Errno 2] No such file or directory: '서울특별시_보행자우선도로_전체.csv'

# 1. 접근성

## 1.1. 데이터 불러오기 및 전처리

### 1.1.0. 행정동 코드

In [ ]:
행정동 = pd.read_excel('./행정동/행정동코드(행안부기준).xlsx')
행정동.drop('시도명', axis=1, inplace=True)
행정동.columns = ['행정동코드', '자치구명', '행정구역명']
행정동.head()
# 하 ㅋㅋ 얜 행정기관코드래

,행정동코드,자치구명,행정구역명
0,11110515,종로구,청운효자동
1,11110530,종로구,사직동
2,11110540,종로구,삼청동
3,11110550,종로구,부암동
4,11110560,종로구,평창동


In [ ]:
행정동2 = pd.read_excel('./행정동/행정동코드(2022.10.기준).xlsx')
행정동2.rename(columns = {'분류번호':'행정동코드'},inplace=True)
행정동2.head()

#pd.merge(left=행정동2, right=행정동행, how="outer", on="행정동코드")
#행정동2.shape, 행정동행.shape
#### 행정동 행안부 데이터로!

,자치구명,행정동코드,행정구역명
0,강남구,11230510,신사동
1,강남구,11230520,논현1동
2,강남구,11230530,논현2동
3,강남구,11230770,압구정동
4,강남구,11230780,청담동


In [ ]:
행정구역분류 = pd.read_excel('./행정동/한국행정구역분류(2022.10.1.기준).xlsx', \
    sheet_name='행정동 및 법정동코드와의 연계표', header=1)
행정구역분류 = 행정구역분류[행정구역분류.시도 == '서울특별시'].drop('시도', axis=1)
행정구역분류

,시군구,행정구역명,행정동\n(행정기관명),법정동,행정구역분류\n(개정_8자리),행정구역분류\n(7자리),행정기관코드,행정기관 생성일,법정동코드,법정동 관할구역\n분할여부,행정동 영문명칭,비고
0,서울특별시,서울특별시,서울특별시,서울특별시,11.0,11.0,1100000000,19880423,1100000000,NaN,Seoul,NaN
1,종로구,종로구,종로구,종로구,11010.0,11010.0,1111000000,19880423,1111000000,NaN,Jongno-gu,NaN
2,종로구,청운효자동,청운효자동,청운동,11010720.0,1101072.0,1111051500,20081101,1111010100,NaN,Cheongunhyoja-dong,NaN
3,종로구,청운효자동,청운효자동,신교동,11010720.0,1101072.0,1111051500,20081101,1111010200,NaN,Cheongunhyoja-dong,NaN
4,종로구,청운효자동,청운효자동,궁정동,11010720.0,1101072.0,1111051500,20081101,1111010300,NaN,Cheongunhyoja-dong,NaN
5,종로구,청운효자동,청운효자동,효자동,11010720.0,1101072.0,1111051500,20081101,1111010400,NaN,Cheongunhyoja-dong,NaN
6,종로구,청운효자동,청운효자동,창성동,11010720.0,1101072.0,1111051500,20081101,1111010500,NaN,Cheongunhyoja-dong,NaN
7,종로구,청운효자동,청운효자동,통인동,11010720.0,1101072.0,1111051500,20081101,1111010800,NaN,Cheongunhyoja-dong,NaN
8,종로구,청운효자동,청운효자동,누상동,11010720.0,1101072.0,1111051500,20081101,1111010900,NaN,Cheongunhyoja-dong,NaN
9,종로구,청운효자동,청운효자동,누하동,11010720.0,1101072.0,1111051500,20081101,1111011000,NaN,Cheongunhyoja-dong,NaN


In [ ]:
행정구역분류.columns

Index(['시군구', '행정구역명', '행정동\n(행정기관명)', '법정동', '행정구역분류\n(개정_8자리)',
       '행정구역분류\n(7자리)', '행정기관코드', '행정기관 생성일', '법정동코드', '법정동 관할구역\n분할여부',
       '행정동 영문명칭', '비고'],
      dtype='object')

In [ ]:
행정구역 = 행정구역분류[['시군구', '행정구역명', '행정구역분류\n(개정_8자리)']] #, '행정기관코드', '법정동코드']]
행정구역 = 행정구역.drop_duplicates()

# 시, 구 행 제거
행정구역 = 행정구역[~행정구역['행정구역명'].str.contains('구$')]
행정구역.drop([0], inplace=True) 

행정구역.columns = ['자치구명', '행정구역명', '행정구역분류'] #, '행정동코드', '법정동코드']
행정구역.행정구역분류 = 행정구역.행정구역분류/10 # 뒤에 0 잘라내기
# 행정구역.행정동코드 = 행정구역.행정동코드/100 # 뒤에 00 잘라내기
# 행정구역.법정동코드 = 행정구역.법정동코드/100 # 뒤에 00 잘라내기

행정구역.행정구역분류 = 행정구역.행정구역분류.astype(int)
# 행정구역.행정동코드 = 행정구역.행정동코드.astype(int)
# 행정구역.법정동코드 = 행정구역.법정동코드.astype(int)

행정구역.head(10)

,자치구명,행정구역명,행정구역분류
2,종로구,청운효자동,1101072
12,종로구,사직동,1101053
24,종로구,삼청동,1101054
31,종로구,부암동,1101055
34,종로구,평창동,1101056
36,종로구,무악동,1101057
37,종로구,교남동,1101058
43,종로구,가회동,1101060
47,종로구,종로1.2.3.4가동,1101061
76,종로구,종로5.6가동,1101063


### 1.1.1. 대중교통

**(1) 버스 승객수**

In [ ]:
버스승객 = pd.read_csv('./접근성/대중교통/서울시 행정동별 버스 총 승차 승객수 정보.csv', encoding='cp949')
print(버스승객.shape)
버스승객.head()

(122960, 27)


,기준_날짜,행정동_ID,버스_승객_수,버스_승객_수_00시,버스_승객_수_01시,버스_승객_수_02시,버스_승객_수_03시,버스_승객_수_04시,버스_승객_수_05시,버스_승객_수_06시,버스_승객_수_07시,버스_승객_수_08시,버스_승객_수_09시,버스_승객_수_10시,버스_승객_수_11시,버스_승객_수_12시,버스_승객_수_13시,버스_승객_수_14시,버스_승객_수_15시,버스_승객_수_16시,버스_승객_수_17시,버스_승객_수_18시,버스_승객_수_19시,버스_승객_수_20시,버스_승객_수_21시,버스_승객_수_22시,버스_승객_수_23시
0,20221018,1120071,18662,54,3,0,0,14,238,600,1565,1561,983,877,912,860,924,1060,1259,1239,1532,1394,998,740,725,706,418
1,20221018,1119075,5393,11,1,0,0,19,57,269,813,978,525,289,218,243,226,201,249,210,277,291,180,118,101,81,36
2,20221018,1119063,9531,19,3,1,3,86,328,381,1097,1543,813,424,363,427,421,360,371,535,512,586,399,294,256,205,104
3,20221018,1118057,20427,45,3,10,8,116,310,633,1738,1721,1025,847,924,970,949,1003,1191,1502,1410,1976,1377,963,784,626,296
4,20221018,1117065,4260,7,0,0,0,46,103,314,726,577,303,224,193,191,162,210,185,222,230,197,123,94,63,63,27


In [ ]:
print(버스승객['기준_날짜'].min(), 버스승객['기준_날짜'].max())
# 20220101 ~ 20221018 -> 10달 간의 데이터 (표준화할 거니까 상관 없겠지?)

print(버스승객['행정동_ID'].nunique())
# 424개 행정동

버스승객.describe()

20220101 20221018
424


,기준_날짜,행정동_ID,버스_승객_수,버스_승객_수_00시,버스_승객_수_01시,버스_승객_수_02시,버스_승객_수_03시,버스_승객_수_04시,버스_승객_수_05시,버스_승객_수_06시,버스_승객_수_07시,버스_승객_수_08시,버스_승객_수_09시,버스_승객_수_10시,버스_승객_수_11시,버스_승객_수_12시,버스_승객_수_13시,버스_승객_수_14시,버스_승객_수_15시,버스_승객_수_16시,버스_승객_수_17시,버스_승객_수_18시,버스_승객_수_19시,버스_승객_수_20시,버스_승객_수_21시,버스_승객_수_22시,버스_승객_수_23시
count,1.229600e+05,1.229600e+05,122960.000000,122960.000000,122960.000000,122960.000000,122960.000000,122960.000000,122960.000000,122960.000000,122960.000000,122960.000000,122960.000000,122960.000000,122960.000000,122960.000000,122960.000000,122960.000000,122960.000000,122960.000000,122960.000000,122960.000000,122960.000000,122960.000000,122960.000000,122960.000000,122960.000000
mean,2.022055e+07,1.113664e+06,9408.744933,55.862549,12.629920,6.187199,5.608694,56.260483,134.234353,261.490037,611.768356,778.720543,542.508295,429.305115,425.870007,470.971755,493.634678,497.217469,558.366070,633.313248,688.060760,785.502838,566.295275,429.269925,423.019169,360.006482,182.641713
std,2.759483e+02,7.402773e+03,7526.110696,96.338970,34.337553,17.687251,16.023510,60.973105,106.338954,195.679802,528.017108,668.104094,405.510872,284.250197,296.358339,331.013658,363.988032,389.780780,459.127263,530.085909,653.001303,839.606659,560.925695,436.519231,459.122746,400.842092,230.925430
min,2.022010e+07,1.101053e+06,63.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.022032e+07,1.107070e+06,4618.000000,12.000000,0.000000,0.000000,0.000000,14.000000,55.000000,115.000000,185.000000,293.000000,276.000000,235.000000,229.000000,255.000000,262.000000,256.000000,274.000000,300.000000,308.000000,300.000000,224.000000,171.000000,158.000000,125.000000,55.000000
50%,2.022053e+07,1.114068e+06,7305.000000,28.000000,4.000000,1.000000,1.000000,35.000000,106.000000,214.000000,485.000000,614.000000,444.000000,360.000000,349.000000,385.000000,397.000000,393.000000,429.000000,479.000000,499.000000,535.000000,388.000000,295.000000,280.000000,234.000000,108.000000
75%,2.022081e+07,1.120318e+06,11845.000000,61.000000,12.000000,6.000000,6.000000,77.000000,184.000000,356.000000,884.000000,1050.000000,687.000000,547.000000,540.000000,597.000000,620.000000,621.000000,703.000000,802.000000,844.000000,950.000000,695.000000,522.250000,505.000000,433.000000,215.000000
max,2.022102e+07,1.125074e+06,75650.000000,2368.000000,1122.000000,563.000000,563.000000,514.000000,910.000000,1388.000000,3957.000000,5664.000000,4284.000000,2649.000000,3139.000000,3495.000000,4301.000000,5069.000000,6188.000000,6188.000000,7348.000000,9951.000000,5761.000000,5135.000000,5705.000000,4416.000000,3912.000000


In [ ]:
버스승객수 = 버스승객.groupby('행정동_ID')['버스_승객_수'].sum()
버스승객수 = pd.DataFrame(버스승객수).reset_index()
버스승객수.columns = ["행정동ID", '버스승객수']
pd.merge(버스승객수, 행정동, left_on="행정동ID", right_on="행정동코드", how="left").head()
# 하 .. 빡쳐 이건 또 법정동코드야;;;;;; 행정동 지옥ㅜㅜ 아놔ㅏㅏㅏㅏㅏ

,행정동ID,버스승객수,행정동코드,자치구명,행정구역명
0,1101053,6971522,NaN,NaN,NaN
1,1101054,1201780,NaN,NaN,NaN
2,1101055,2550275,NaN,NaN,NaN
3,1101056,2325866,NaN,NaN,NaN
4,1101057,565384,NaN,NaN,NaN


In [ ]:
행정동별_버스승객수 = pd.merge(버스승객수, 행정구역, left_on="행정동ID", right_on="행정구역분류", how="left")
행정동별_버스승객수 = 행정동별_버스승객수.drop('행정구역분류', axis=1)
행정동별_버스승객수.head()

,행정동ID,버스승객수,자치구명,행정구역명
0,1101053,6971522,종로구,사직동
1,1101054,1201780,종로구,삼청동
2,1101055,2550275,종로구,부암동
3,1101056,2325866,종로구,평창동
4,1101057,565384,종로구,무악동


**(2) 지하철 승객수**

In [ ]:
지하철승객 = pd.read_csv('./접근성/대중교통/서울시 행정동별 지하철 총 승차 승객수 정보.csv', encoding='cp949')
지하철승객.head()

,기준_날짜,행정동_ID,지하철_승객_수,지하철_승객_수_00시,지하철_승객_수_01시,지하철_승객_수_02시,지하철_승객_수_03시,지하철_승객_수_04시,지하철_승객_수_05시,지하철_승객_수_06시,지하철_승객_수_07시,지하철_승객_수_08시,지하철_승객_수_09시,지하철_승객_수_10시,지하철_승객_수_11시,지하철_승객_수_12시,지하철_승객_수_13시,지하철_승객_수_14시,지하철_승객_수_15시,지하철_승객_수_16시,지하철_승객_수_17시,지하철_승객_수_18시,지하철_승객_수_19시,지하철_승객_수_20시,지하철_승객_수_21시,지하철_승객_수_22시,지하철_승객_수_23시
0,20221018,1120071,9345.0,34.0,0.0,0.0,0.0,0.0,130.0,256.0,757.0,964.0,588.0,455.0,441.0,412.0,461.0,447.0,587.0,665.0,952.0,683.0,436.0,322.0,317.0,294.0,144.0
1,20221018,1119075,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20221018,1119063,11194.0,7.0,0.0,0.0,0.0,17.0,406.0,489.0,1570.0,2221.0,959.0,525.0,345.0,432.0,372.0,286.0,281.0,412.0,661.0,1052.0,433.0,245.0,205.0,194.0,82.0
3,20221018,1118057,7132.0,1.0,0.0,0.0,0.0,1.0,193.0,501.0,1470.0,1084.0,667.0,382.0,286.0,299.0,226.0,207.0,237.0,266.0,362.0,471.0,175.0,124.0,93.0,68.0,19.0
4,20221018,1117065,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
지하철승객.describe()

,기준_날짜,행정동_ID,지하철_승객_수,지하철_승객_수_00시,지하철_승객_수_01시,지하철_승객_수_02시,지하철_승객_수_03시,지하철_승객_수_04시,지하철_승객_수_05시,지하철_승객_수_06시,지하철_승객_수_07시,지하철_승객_수_08시,지하철_승객_수_09시,지하철_승객_수_10시,지하철_승객_수_11시,지하철_승객_수_12시,지하철_승객_수_13시,지하철_승객_수_14시,지하철_승객_수_15시,지하철_승객_수_16시,지하철_승객_수_17시,지하철_승객_수_18시,지하철_승객_수_19시,지하철_승객_수_20시,지하철_승객_수_21시,지하철_승객_수_22시,지하철_승객_수_23시
count,1.242320e+05,1.242320e+05,64215.000000,64215.000000,64215.00000,64215.000000,64215.000000,64215.000000,64215.000000,64215.000000,64215.000000,64215.000000,64215.000000,64215.000000,64215.000000,64215.000000,64215.000000,64215.000000,64215.000000,64215.000000,64215.000000,64215.000000,64215.000000,64215.000000,64215.000000,64215.000000,64215.000000
mean,2.022045e+07,1.113664e+06,19494.777684,18.494635,0.03781,0.001962,0.000016,6.582527,261.251078,530.553702,1392.969337,1731.007802,1070.553251,792.483610,810.693062,901.189722,935.522946,950.110130,1069.707732,1264.233684,1702.567905,2217.614747,1137.698497,910.159620,887.784054,656.612988,246.946866
std,9.807174e+02,7.402772e+03,19909.340502,70.407246,0.65320,0.070565,0.003946,20.548061,287.878689,515.188390,1504.997075,1878.562369,1010.265898,656.177193,712.416365,819.536523,908.675982,992.696978,1191.601020,1429.238241,2345.797580,3556.169321,1699.620829,1364.793828,1408.455245,1121.970703,492.578997
min,2.021113e+07,1.101053e+06,175.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,2.000000,2.000000,5.000000,2.000000,6.000000,5.000000,7.000000,9.000000,8.000000,13.000000,10.000000,9.000000,6.000000,1.000000,1.000000,0.000000,0.000000,0.000000
25%,2.022031e+07,1.107070e+06,6761.000000,0.000000,0.00000,0.000000,0.000000,0.000000,90.000000,186.000000,324.000000,491.000000,432.000000,341.000000,323.000000,352.000000,343.000000,323.000000,338.000000,390.000000,416.000000,381.000000,234.000000,179.000000,156.000000,110.000000,34.000000
50%,2.022052e+07,1.114068e+06,12456.000000,0.000000,0.00000,0.000000,0.000000,1.000000,176.000000,377.000000,897.000000,1163.000000,782.000000,604.000000,587.000000,633.000000,626.000000,596.000000,634.000000,744.000000,846.000000,908.000000,494.000000,368.000000,341.000000,244.000000,81.000000
75%,2.022081e+07,1.120318e+06,24608.000000,6.000000,0.00000,0.000000,0.000000,4.000000,331.000000,677.000000,1928.000000,2356.000000,1364.500000,1042.000000,1056.000000,1173.000000,1208.000000,1216.000000,1343.500000,1575.500000,1910.000000,2175.000000,1217.500000,1010.000000,962.000000,713.000000,248.000000
max,2.022102e+07,1.125074e+06,152516.000000,1941.000000,65.00000,11.000000,1.000000,273.000000,2903.000000,3791.000000,11656.000000,14546.000000,8176.000000,4830.000000,6187.000000,9831.000000,7624.000000,10042.000000,14706.000000,16724.000000,20939.000000,26256.000000,12697.000000,14460.000000,20218.000000,22276.000000,10117.000000


In [ ]:
print(지하철승객['기준_날짜'].min(), 지하철승객['기준_날짜'].max())
# 20220101 ~ 20221018 -> 10달 간의 데이터 (표준화할 거니까 상관 없겠지?)

print(지하철승객['행정동_ID'].nunique())
# 424개 행정동

지하철승객.describe()

20211130 20221018
424


,기준_날짜,행정동_ID,지하철_승객_수,지하철_승객_수_00시,지하철_승객_수_01시,지하철_승객_수_02시,지하철_승객_수_03시,지하철_승객_수_04시,지하철_승객_수_05시,지하철_승객_수_06시,지하철_승객_수_07시,지하철_승객_수_08시,지하철_승객_수_09시,지하철_승객_수_10시,지하철_승객_수_11시,지하철_승객_수_12시,지하철_승객_수_13시,지하철_승객_수_14시,지하철_승객_수_15시,지하철_승객_수_16시,지하철_승객_수_17시,지하철_승객_수_18시,지하철_승객_수_19시,지하철_승객_수_20시,지하철_승객_수_21시,지하철_승객_수_22시,지하철_승객_수_23시
count,1.242320e+05,1.242320e+05,64215.000000,64215.000000,64215.00000,64215.000000,64215.000000,64215.000000,64215.000000,64215.000000,64215.000000,64215.000000,64215.000000,64215.000000,64215.000000,64215.000000,64215.000000,64215.000000,64215.000000,64215.000000,64215.000000,64215.000000,64215.000000,64215.000000,64215.000000,64215.000000,64215.000000
mean,2.022045e+07,1.113664e+06,19494.777684,18.494635,0.03781,0.001962,0.000016,6.582527,261.251078,530.553702,1392.969337,1731.007802,1070.553251,792.483610,810.693062,901.189722,935.522946,950.110130,1069.707732,1264.233684,1702.567905,2217.614747,1137.698497,910.159620,887.784054,656.612988,246.946866
std,9.807174e+02,7.402772e+03,19909.340502,70.407246,0.65320,0.070565,0.003946,20.548061,287.878689,515.188390,1504.997075,1878.562369,1010.265898,656.177193,712.416365,819.536523,908.675982,992.696978,1191.601020,1429.238241,2345.797580,3556.169321,1699.620829,1364.793828,1408.455245,1121.970703,492.578997
min,2.021113e+07,1.101053e+06,175.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,2.000000,2.000000,5.000000,2.000000,6.000000,5.000000,7.000000,9.000000,8.000000,13.000000,10.000000,9.000000,6.000000,1.000000,1.000000,0.000000,0.000000,0.000000
25%,2.022031e+07,1.107070e+06,6761.000000,0.000000,0.00000,0.000000,0.000000,0.000000,90.000000,186.000000,324.000000,491.000000,432.000000,341.000000,323.000000,352.000000,343.000000,323.000000,338.000000,390.000000,416.000000,381.000000,234.000000,179.000000,156.000000,110.000000,34.000000
50%,2.022052e+07,1.114068e+06,12456.000000,0.000000,0.00000,0.000000,0.000000,1.000000,176.000000,377.000000,897.000000,1163.000000,782.000000,604.000000,587.000000,633.000000,626.000000,596.000000,634.000000,744.000000,846.000000,908.000000,494.000000,368.000000,341.000000,244.000000,81.000000
75%,2.022081e+07,1.120318e+06,24608.000000,6.000000,0.00000,0.000000,0.000000,4.000000,331.000000,677.000000,1928.000000,2356.000000,1364.500000,1042.000000,1056.000000,1173.000000,1208.000000,1216.000000,1343.500000,1575.500000,1910.000000,2175.000000,1217.500000,1010.000000,962.000000,713.000000,248.000000
max,2.022102e+07,1.125074e+06,152516.000000,1941.000000,65.00000,11.000000,1.000000,273.000000,2903.000000,3791.000000,11656.000000,14546.000000,8176.000000,4830.000000,6187.000000,9831.000000,7624.000000,10042.000000,14706.000000,16724.000000,20939.000000,26256.000000,12697.000000,14460.000000,20218.000000,22276.000000,10117.000000


In [ ]:
지하철승객수 = 지하철승객.groupby('행정동_ID')['지하철_승객_수'].sum()
지하철승객수 = pd.DataFrame(지하철승객수).reset_index()
지하철승객수.columns = ["행정동ID", '지하철승객수']

행정동별_지하철승객수 = pd.merge(지하철승객수, 행정구역, left_on="행정동ID", right_on="행정구역분류", how="left")
행정동별_지하철승객수 = 행정동별_지하철승객수.drop('행정구역분류', axis=1)
행정동별_지하철승객수.head() # 지하철역 없는 동에는 0.. 내가 머 잘못한 줄;;

,행정동ID,지하철승객수,자치구명,행정구역명
0,1101053,13394787.0,종로구,사직동
1,1101054,0.0,종로구,삼청동
2,1101055,0.0,종로구,부암동
3,1101056,0.0,종로구,평창동
4,1101057,0.0,종로구,무악동


**(3) 버스 위치**

In [ ]:
버스위치 = pd.read_csv('./접근성/대중교통/서울시버스정류소좌표데이터(2022.08.24).csv', encoding='cp949')
버스위치.drop(['NODE_ID', 'ARS-ID'], axis=1, inplace=True)
버스위치.columns = ['정류소명', '경도', '위도']
버스위치.head()

,정류소명,경도,위도
0,종로2가사거리,126.987786,37.569764
1,창경궁.서울대학교병원,126.996520,37.579179
2,명륜3가.성대입구,126.998290,37.582709
3,종로2가.삼일교,126.987507,37.568582
4,혜화동로터리.여운형활동터,127.001694,37.586230


역지오코딩
참고 : https://gis.stackexchange.com/questions/427341/reverse-geocoding-of-pandas-dataframe-with-lat-long-columns

In [ ]:
# 가입 없이 좌표->주소 변환 (역지오코딩)
from geopy.geocoders import Nominatim
from geopy.point import Point

def geocoding_reverse(lat, lon): 
    geolocoder = Nominatim(user_agent = 'South Korea', timeout=None)
    address = geolocoder.reverse(Point(lat, lon))

    return address

In [ ]:
버스위치['address'] = np.vectorize(geocoding_reverse)(버스위치['위도'], 버스위치['경도'])

In [ ]:
버스위치.to_csv("./접근성/대중교통/버스정류장주소변환.csv")

**(4) 지하철 위치**

In [ ]:
지하철위치 = pd.read_csv('./접근성/대중교통/서울시역사마스터정보.csv', header=None)
지하철위치.columns = ['호선', '역사명', '역사ID', '위도', '경도']
지하철위치.head()

,호선,역사명,역사ID,위도,경도
0,1호선,녹양,1908.0,37.759380,127.042292
1,1호선,남영,1002.0,37.541021,126.971300
2,1호선,용산,1003.0,37.529849,126.964561
3,1호선,노량진,1004.0,37.514219,126.942454
4,1호선,대방,1005.0,37.513342,126.926382


In [ ]:
지하철위치['address'] = np.vectorize(geocoding_reverse)(지하철위치['위도'], 지하철위치['경도'])

In [ ]:
지하철위치.to_csv("./접근성/대중교통/지하철역주소변환.csv")

In [ ]:
지하철위치 = pd.read_csv("./접근성/대중교통/지하철역주소변환.csv", index_col=0)
지하철위치.head()

,호선,역사명,역사ID,위도,경도,address
0,1호선,녹양,1908.0,37.759380,127.042292,"757, 평화로, 의정부시, 경기도, 11684, 대한민국"
1,1호선,남영,1002.0,37.541021,126.971300,"2, 한강대로77길, 남영동, 용산구, 서울, 04314, 대한민국"
2,1호선,용산,1003.0,37.529849,126.964561,"55, 한강대로23길, 한강로동, 한강로3가, 서울, 04377, 대한민국"
3,1호선,노량진,1004.0,37.514219,126.942454,"3, 노량진로, 노량진1동, 동작구, 서울, 06922, 대한민국"
4,1호선,대방,1005.0,37.513342,126.926382,"2, 대방지하차도, 대방동, 동작구, 서울, 07354, 대한민국"


In [ ]:
# 저 address column 머야 ㅠㅠ 뜯어보기...
지하철위치.address.str[0:].head() # 다 나옴
#지하철위치.address.str[1].head() # 위도 경도만 나옴
#지하철위치.address.str[0].head()

1        2, 한강대로77길, 남영동, 용산구, 서울, 04314, 대한민국
2    55, 한강대로23길, 한강로동, 한강로3가, 서울, 04377, 대한민국
3         3, 노량진로, 노량진1동, 동작구, 서울, 06922, 대한민국
4         2, 대방지하차도, 대방동, 동작구, 서울, 07354, 대한민국
5      6, 영신로19길, 영등포본동, 영등포구, 서울, 07313, 대한민국
Name: address, dtype: object

In [ ]:
지하철위치.address = 지하철위치.address.str[0:] # object
지하철위치 = 지하철위치[지하철위치.address.str.contains('서울')]
지하철위치.tail()

,호선,역사명,역사ID,위도,경도,address
654,우이신설경전철,성신여대입구,4711.0,37.592624,127.016403,"성신여대입구, 동소문로, 동선동, 성북구, 서울, 02847, 대한민국"
655,우이신설경전철,보문,4712.0,37.585286,127.019381,"L12, 보문로, Bomundong, 성북구, 서울, 02858, 대한민국"
656,우이신설경전철,신설동,4713.0,37.575297,127.025087,"천호대로, 용신동, 동대문구, 서울, 02586, 대한민국"
657,우이신설경전철,화계,4705.0,37.634802,127.017519,"삼양로, 수유동, 강북구, 서울, 01100, 대한민국"
658,우이신설경전철,삼양,4706.0,37.627165,127.018152,"삼양, 삼양로, 삼양동, 강북구, 서울, 01121, 대한민국"


In [ ]:
지하철위치.address.str.split(', ').str[2]
# 동 아닌 행도..

1                  남영동
2                 한강로동
3                노량진1동
4                  대방동
5                영등포본동
6                  경인로
7                 휘경1동
8                 이문2동
9                 이문1동
10                 월계동
11                월계2동
12                 월계동
13                  창동
14                영등포동
15                 월계동
22                신도림동
23     서울디지털국가산업단지 3단지
24                시흥1동
34                독산1동
47                개봉2동
48                오류2동
61                구로1동
66                 수궁동
72                 방학동
73                도봉2동
74                도봉2동
86                 성북동
87                종로3가
88                  종로
89                동대문구
90                동대문구
91                전농1동
92                숭인2동
95                 회현동
96                 소공동
97                 성북동
98                강변역로
99                 오금로
100               잠실6동
101               잠실2동
102               올림픽로
103               테헤란로
104               대치4동
105        

In [ ]:
string = "2, 한강대로77길, 남영동, 용산구, 서울, 04314, 대한민국"

try:
    found = re.search('/[ㄱ-ㅎ|가-힣|0-9]동\,\s/g', string).group(1)
    print(found)
except AttributeError:
    pass

In [ ]:
text = "2, 한강대로77길, 남영동, 용산구, 서울, 04314, 대한민국"

m = re.search(', (.+?)동', text)
if m:
    found = m.group(1)
found

'한강대로77길, 남영'

In [ ]:
지하철위치.shape

(390, 6)

In [ ]:
m = []
found = []

for text in 지하철위치.address.to_list() : 
    m = re.search(', (.+?)동', text)
    if m:
        found = m.group(1)
found

TypeError: list indices must be integers or slices, not str

In [ ]:
지하철리스트 = 지하철위치.address.str.split(", ").to_list
#지하철위치.address.str.endswith("동")
지하철리스트

#리스트 컴프리헨션 사용
#matching = [s for s in 지하철리스트 if "동" in s] 
#print(matching)

<bound method IndexOpsMixin.tolist of 1                [2, 한강대로77길, 남영동, 용산구, 서울, 04314, 대한민국]
2            [55, 한강대로23길, 한강로동, 한강로3가, 서울, 04377, 대한민국]
3                 [3, 노량진로, 노량진1동, 동작구, 서울, 06922, 대한민국]
4                 [2, 대방지하차도, 대방동, 동작구, 서울, 07354, 대한민국]
5              [6, 영신로19길, 영등포본동, 영등포구, 서울, 07313, 대한민국]
6           [신도림역, 688, 경인로, 신도림동, 구로구, 서울, 08209, 대한민국]
7             [196, 회기로31길, 휘경1동, 동대문구, 서울, 02444, 대한민국]
8               [1, 휘경로17길, 이문2동, 동대문구, 서울, 02403, 대한민국]
9            [신이문역, 이문고가차도, 이문1동, 동대문구, 서울, 02407, 대한민국]
10                 [광운대, 석계로, 월계동, 노원구, 서울, 01906, 대한민국]
11               [1, 월계로53길, 월계2동, 노원구, 서울, 01885, 대한민국]
12                 [1, 마들로5길, 월계동, 노원구, 서울, 01867, 대한민국]
13               [창동, 마들로11가길, 창동, 도봉구, 서울, 01418, 대한민국]
14       [Dunkin', 영등포로57길, 영등포동, 영등포구, 서울, 07316, 대한민국]
15                [2, 화랑로45길, 월계동, 노원구, 서울, 02781, 대한민국]
22             [GS 네오텍, 경인로, 신도림동, 구로구, 서울, 08213, 대한민국]
23     [우림라이온스밸리 A동, 가산디지털1로, 서울디지털국가산업단지 3단지, 가산동

**(4)_1 지하철 위치_주은**

In [ ]:
df = pd.read_csv("data/지하철역주소변환.csv", index_col=0)
df.head()

,호선,역사명,역사ID,위도,경도,address
0,1호선,녹양,1908.0,37.759380,127.042292,"757, 평화로, 의정부시, 경기도, 11684, 대한민국"
1,1호선,남영,1002.0,37.541021,126.971300,"2, 한강대로77길, 남영동, 용산구, 서울, 04314, 대한민국"
2,1호선,용산,1003.0,37.529849,126.964561,"55, 한강대로23길, 한강로동, 한강로3가, 서울, 04377, 대한민국"
3,1호선,노량진,1004.0,37.514219,126.942454,"3, 노량진로, 노량진1동, 동작구, 서울, 06922, 대한민국"
4,1호선,대방,1005.0,37.513342,126.926382,"2, 대방지하차도, 대방동, 동작구, 서울, 07354, 대한민국"


In [ ]:
df = df[df['address'].str.contains('서울')] ## df[조건식]
df.head()

,호선,역사명,역사ID,위도,경도,address
1,1호선,남영,1002.0,37.541021,126.971300,"2, 한강대로77길, 남영동, 용산구, 서울, 04314, 대한민국"
2,1호선,용산,1003.0,37.529849,126.964561,"55, 한강대로23길, 한강로동, 한강로3가, 서울, 04377, 대한민국"
3,1호선,노량진,1004.0,37.514219,126.942454,"3, 노량진로, 노량진1동, 동작구, 서울, 06922, 대한민국"
4,1호선,대방,1005.0,37.513342,126.926382,"2, 대방지하차도, 대방동, 동작구, 서울, 07354, 대한민국"
5,1호선,영등포,1006.0,37.515504,126.907628,"6, 영신로19길, 영등포본동, 영등포구, 서울, 07313, 대한민국"


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 390 entries, 1 to 658
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   호선       390 non-null    object 
 1   역사명      390 non-null    object 
 2   역사ID     387 non-null    float64
 3   위도       390 non-null    float64
 4   경도       390 non-null    float64
 5   address  390 non-null    object 
dtypes: float64(3), object(3)
memory usage: 21.3+ KB


In [ ]:
address_list = df['address'].values.tolist()
address_list = ",".join(address_list)
address_list = address_list.split(',')
address_list

['2',
 ' 한강대로77길',
 ' 남영동',
 ' 용산구',
 ' 서울',
 ' 04314',
 ' 대한민국',
 '55',
 ' 한강대로23길',
 ' 한강로동',
 ' 한강로3가',
 ' 서울',
 ' 04377',
 ' 대한민국',
 '3',
 ' 노량진로',
 ' 노량진1동',
 ' 동작구',
 ' 서울',
 ' 06922',
 ' 대한민국',
 '2',
 ' 대방지하차도',
 ' 대방동',
 ' 동작구',
 ' 서울',
 ' 07354',
 ' 대한민국',
 '6',
 ' 영신로19길',
 ' 영등포본동',
 ' 영등포구',
 ' 서울',
 ' 07313',
 ' 대한민국',
 '신도림역',
 ' 688',
 ' 경인로',
 ' 신도림동',
 ' 구로구',
 ' 서울',
 ' 08209',
 ' 대한민국',
 '196',
 ' 회기로31길',
 ' 휘경1동',
 ' 동대문구',
 ' 서울',
 ' 02444',
 ' 대한민국',
 '1',
 ' 휘경로17길',
 ' 이문2동',
 ' 동대문구',
 ' 서울',
 ' 02403',
 ' 대한민국',
 '신이문역',
 ' 이문고가차도',
 ' 이문1동',
 ' 동대문구',
 ' 서울',
 ' 02407',
 ' 대한민국',
 '광운대',
 ' 석계로',
 ' 월계동',
 ' 노원구',
 ' 서울',
 ' 01906',
 ' 대한민국',
 '1',
 ' 월계로53길',
 ' 월계2동',
 ' 노원구',
 ' 서울',
 ' 01885',
 ' 대한민국',
 '1',
 ' 마들로5길',
 ' 월계동',
 ' 노원구',
 ' 서울',
 ' 01867',
 ' 대한민국',
 '창동',
 ' 마들로11가길',
 ' 창동',
 ' 도봉구',
 ' 서울',
 ' 01418',
 ' 대한민국',
 "Dunkin'",
 ' 영등포로57길',
 ' 영등포동',
 ' 영등포구',
 ' 서울',
 ' 07316',
 ' 대한민국',
 '2',
 ' 화랑로45길',
 ' 월계동',
 ' 노원구',
 ' 서울',
 ' 0278

In [ ]:
len(address_list)

15760

In [ ]:
matching = [i for i in address_list if "동" in i]
print(matching)

[' 남영동', ' 한강로동', ' 노량진1동', ' 동작구', ' 대방동', ' 동작구', ' 영등포본동', ' 신도림동', ' 휘경1동', ' 동대문구', ' 이문2동', ' 동대문구', ' 이문1동', ' 동대문구', ' 월계동', ' 월계2동', ' 월계동', '창동', ' 창동', ' 영등포동', ' 월계동', ' 신도림동', '우림라이온스밸리 A동', ' 가산동', ' 시흥1동', ' 독산1동', ' 개봉2동', ' 오류2동', ' 구로1동', ' 수궁동', ' 방학동', ' 도봉2동', ' 도봉동', ' 도봉2동', ' 동소문로', ' 성북동', ' 종로5·6가동', '흥인지문 | 동대문', ' 종로5·6가동', ' 용신동', ' 동대문구', ' 용신동', ' 동대문구', ' 전농1동', ' 동대문구', ' 숭인2동', ' 회현동', ' 소공동', ' 동소문로', ' 성북동', ' 구의3동', ' 행정동', ' 잠실6동', ' 잠실2동', '종합운동장', ' 잠실7동', ' 대치2동', ' 대치4동', ' 역삼동', ' 역삼동', ' 서초동', ' 방배3동', ' 남현동', ' 인헌동', ' 행운동', ' 청룡동', ' 서원동', ' 조원동', ' 구로3동', ' 신도림동', ' 문래동', ' 당산1동', ' 용답동', ' 성동구', ' 신정3동', ' 명동', ' 을지로동', ' 을지로동', '광희동', ' 동대문패션타운', ' 광희동', ' 황학동', ' 사근동', ' 성동구', ' 성수1가1동', ' 성동구', ' 성수2가3동', ' 성동구', ' 동일로22길', ' 화양동', ' 자양2동', ' 당산2동', ' 합정동', ' 서교동', ' 신촌동', ' 아현동', ' 아현동', ' 충현동', ' 용답동', ' 성동구', ' 화곡1동', ' 소공동', ' 서초동', '행당1동', ' 성동구', ' 왕십리2동', ' 성동구', ' 대림3동', ' 용신동', ' 동대문구', ' 신도림동', ' 목동로3길', ' 신정7동', '동대문구청', ' 용

In [ ]:
matching = pd.DataFrame(zip(matching))
matching.head()

,0
0,남영동
1,한강로동
2,노량진1동
3,동작구
4,대방동


In [ ]:
matching[1] = matching[0].str.endswith('동')
matching.head()
#matching.head()

,0,1
0,남영동,True
1,한강로동,True
2,노량진1동,True
3,동작구,False
4,대방동,True


In [ ]:
matching_final =matching[matching[1]==True]
matching_final.head()

,0,1
0,남영동,True
1,한강로동,True
2,노량진1동,True
4,대방동,True
6,영등포본동,True


In [ ]:
matching_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 400 entries, 0 to 520
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       400 non-null    object
 1   1       400 non-null    bool  
dtypes: bool(1), object(1)
memory usage: 6.6+ KB


In [ ]:
matching_final = matching_final.reset_index()
matching_final.head(20)


,index,0,1
0,0,남영동,True
1,1,한강로동,True
2,2,노량진1동,True
3,4,대방동,True
4,6,영등포본동,True
5,7,신도림동,True
6,8,휘경1동,True
7,10,이문2동,True
8,12,이문1동,True
9,14,월계동,True


In [ ]:
matching_final.to_csv('dong_subway.csv')

In [ ]:
#삭제 후 합치기
df_1 = pd.read_csv('dong_subway.csv', encoding = 'cp949')
df_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 390 entries, 0 to 389
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  390 non-null    int64 
 1   index       390 non-null    int64 
 2   0           390 non-null    object
 3   1           390 non-null    bool  
dtypes: bool(1), int64(2), object(1)
memory usage: 9.6+ KB


In [ ]:
df_final_subway = pd.concat([df,df_1], axis = 1) 
df_final_subway =df_final_subway.drop(['Unnamed: 0','index','1'],axis = 1)
df_final_subway.head()

,호선,역사명,역사ID,위도,경도,address,0
1,1호선,남영,1002.0,37.541021,126.971300,"2, 한강대로77길, 남영동, 용산구, 서울, 04314, 대한민국",한강로동
2,1호선,용산,1003.0,37.529849,126.964561,"55, 한강대로23길, 한강로동, 한강로3가, 서울, 04377, 대한민국",노량진1동
3,1호선,노량진,1004.0,37.514219,126.942454,"3, 노량진로, 노량진1동, 동작구, 서울, 06922, 대한민국",대방동
4,1호선,대방,1005.0,37.513342,126.926382,"2, 대방지하차도, 대방동, 동작구, 서울, 07354, 대한민국",영등포본동
5,1호선,영등포,1006.0,37.515504,126.907628,"6, 영신로19길, 영등포본동, 영등포구, 서울, 07313, 대한민국",신도림동


In [ ]:
df_final_subway.to_csv('subway_dong_final.csv')

**(4)-2. 버스**

In [ ]:
df2 = pd.read_csv("data/버스정류장주소변환.csv", index_col=0)
df2.head()

,정류소명,경도,위도,address
0,종로2가사거리,126.987786,37.569764,"서울 한양도성, 동소문로, 성북동, 성북구, 서울, 02847, 대한민국"
1,창경궁.서울대학교병원,126.996520,37.579179,"창경궁, 185, 창경궁로, 종로1·2·3·4가동, 종로구, 서울, 03075, 대한민국"
2,명륜3가.성대입구,126.998290,37.582709,"Subway, 창경궁로, 혜화동, 종로구, 서울, 03075, 대한민국"
3,종로2가.삼일교,126.987507,37.568582,"서울한양도성, 동소문로, 성북동, 성북구, 서울, 02847, 대한민국"
4,혜화동로터리.여운형활동터,127.001694,37.586230,"혜화동로터리.여운형활동터, 창경궁로, 혜화동, 종로구, 서울, 03075, 대한민국"


In [ ]:
df2 = df2[df2['address'].str.contains('서울')] ## df[조건식]
df2.head()

,정류소명,경도,위도,address
0,종로2가사거리,126.987786,37.569764,"서울 한양도성, 동소문로, 성북동, 성북구, 서울, 02847, 대한민국"
1,창경궁.서울대학교병원,126.996520,37.579179,"창경궁, 185, 창경궁로, 종로1·2·3·4가동, 종로구, 서울, 03075, 대한민국"
2,명륜3가.성대입구,126.998290,37.582709,"Subway, 창경궁로, 혜화동, 종로구, 서울, 03075, 대한민국"
3,종로2가.삼일교,126.987507,37.568582,"서울한양도성, 동소문로, 성북동, 성북구, 서울, 02847, 대한민국"
4,혜화동로터리.여운형활동터,127.001694,37.586230,"혜화동로터리.여운형활동터, 창경궁로, 혜화동, 종로구, 서울, 03075, 대한민국"


In [ ]:
df2.shape

(11129, 4)

In [ ]:
address_list = df2['address'].values.tolist()
address_list = ",".join(address_list)
address_list = address_list.split(',')
address_list

['서울 한양도성',
 ' 동소문로',
 ' 성북동',
 ' 성북구',
 ' 서울',
 ' 02847',
 ' 대한민국',
 '창경궁',
 ' 185',
 ' 창경궁로',
 ' 종로1·2·3·4가동',
 ' 종로구',
 ' 서울',
 ' 03075',
 ' 대한민국',
 'Subway',
 ' 창경궁로',
 ' 혜화동',
 ' 종로구',
 ' 서울',
 ' 03075',
 ' 대한민국',
 '서울한양도성',
 ' 동소문로',
 ' 성북동',
 ' 성북구',
 ' 서울',
 ' 02847',
 ' 대한민국',
 '혜화동로터리.여운형활동터',
 ' 창경궁로',
 ' 혜화동',
 ' 종로구',
 ' 서울',
 ' 03075',
 ' 대한민국',
 '서대문역 3번 출구',
 ' 새문안로',
 ' 교남동',
 ' 종로구',
 ' 서울',
 ' 03735',
 ' 대한민국',
 '서울역사박물관.경희궁앞',
 ' 새문안로',
 ' 사직동',
 ' 종로구',
 ' 서울',
 ' 03177',
 ' 대한민국',
 '서울역사박물관.경희궁앞',
 ' 새문안로',
 ' 사직동',
 ' 종로구',
 ' 서울',
 ' 03177',
 ' 대한민국',
 '에스호텔',
 ' 새문안로',
 ' 사직동',
 ' 종로구',
 ' 서울',
 ' 03186',
 ' 대한민국',
 '교보생명빌딩',
 ' 1',
 ' 종로',
 ' 종로3가',
 ' 종로1·2·3·4가동',
 ' 종로구',
 ' 서울',
 ' 03154',
 ' 대한민국',
 '종로1가',
 ' 종로 자전거 전용로',
 ' 종로1·2·3·4가동',
 ' 종로구',
 ' 서울',
 ' 03188',
 ' 대한민국',
 '서울한양도성',
 ' 동소문로',
 ' 성북동',
 ' 성북구',
 ' 서울',
 ' 02847',
 ' 대한민국',
 '서울한양도성',
 ' 동소문로',
 ' 성북동',
 ' 성북구',
 ' 서울',
 ' 02847',
 ' 대한민국',
 '서울한양도성',
 ' 동소문로',
 ' 성북동',
 ' 성북구',
 ' 서울'

In [ ]:
matching = [i for i in address_list if "동" in i]
print(matching)

In [ ]:
matching = pd.DataFrame(zip(matching))
matching.head()

,0
0,동소문로
1,성북동
2,종로1·2·3·4가동
3,혜화동
4,동소문로


In [55]:
matching[1] = matching[0].str.endswith('동')
matching.head()
#matching.head()

,0,1
0,동소문로,False
1,성북동,True
2,종로1·2·3·4가동,True
3,혜화동,True
4,동소문로,False


In [56]:
matching_final =matching[matching[1]==True]
matching_final.head()

,0,1
1,성북동,True
2,종로1·2·3·4가동,True
3,혜화동,True
5,성북동,True
7,혜화동,True


In [57]:
matching_final.shape

(11531, 2)

In [58]:
matching_final.to_csv('dong_bus.csv')

### 1.1.2. 자전거

(1) 위치

In [303]:
자전거위치 = pd.read_csv('./접근성/자전거/서울시 공공자전거 대여소 정보.csv', encoding='cp949')
자전거위치.columns = ['대여소번호', '대여소명', '자치구', '상세주소', '위도', '경도', '설치시기', '설치형태LCD', '설치형태QR', '운영방식']
자전거위치.drop([0,1,2,3], inplace=True)
자전거위치.head()

,대여소번호,대여소명,자치구,상세주소,위도,경도,설치시기,설치형태LCD,설치형태QR,운영방식
4,102.0,망원역 1번출구 앞,마포구,서울특별시 마포구 월드컵로 72,37.5556488,126.9106293,2022-05-03,NaN,15,QR
5,103.0,망원역 2번출구 앞,마포구,서울특별시 마포구 월드컵로 79,37.55495071,126.9108353,2022-05-03,NaN,14,QR
6,104.0,합정역 1번출구 앞,마포구,서울특별시 마포구 양화로 59,37.55062866,126.9149857,2022-05-12,NaN,13,QR
7,105.0,합정역 5번출구 앞,마포구,서울특별시 마포구 양화로 48,37.55000687,126.9148254,2015-09-06,NaN,5,QR
8,106.0,합정역 7번출구 앞,마포구,서울특별시 마포구 독막로 4,37.54864502,126.9128265,2022-05-12,NaN,12,QR


In [81]:
자전거위치.describe()

,대여소번호
count,2653.000000
mean,2183.524312
std,1464.033115
min,102.000000
25%,950.000000
50%,1928.000000
75%,3545.000000
max,5855.000000


**(2) 도로**

자전거도로 = 자전거전용도로 + 자전거보행자겸용도로 + 자전거전용차로 + 자전거우선도로

In [311]:
자전거도로 = pd.read_csv('./접근성/자전거/서울시 자전거도로 현황 통계.csv')
자전거도로 = 자전거도로[['지역별(3)', '2021']] # 2021 자전거도로 노선수 소계
자전거도로.drop([0,1,2,3,4,30,31,32,33], inplace=True)
자전거도로.columns = ['자치구', '노선수']
자전거도로.head()

,자치구,노선수
5,종로구,42
6,중구,21
7,용산구,26
8,성동구,30
9,광진구,49


### 1.1.3. 주차장

geopandas 설치 엉엉 ㅜㅜ!!

In [ ]:
노외주차장 = './접근성/주차장/B410. 서울시 노외주차장 공간데이터/2. 파일데이터/2019/TL_PRKPLCE_ST_2019.shp'

In [ ]:
노상주차장 = './접근성/주차장/B411. 서울시 노상주차장 공간데이터/2. 파일데이터/2019/TL_STKLT_ST_2019.shp'

## 1.2. 접근성 지표

우선 아래는 주은 언니 코드

In [ ]:
from sklearn.preprocessing import MinMaxScaler
df_safety_test = df_safety.drop(['Gu'],axis = 1)
min_max_scaler = MinMaxScaler()
fitted = min_max_scaler.fit(df_safety_test)
df_safety_test_s = min_max_scaler.transform(df_safety_test)
df_safety_test_s = pd.DataFrame(df_safety_test_s, columns=(df_safety_test).columns, index=list(df_safety_test.index.values))
df_safety_test_s.head()

## 1.3. KMO

In [ ]:
# Bartlett test
from factor_analyzer.factor_analyzer import calculate_bartlett_sphericity
chi_square_value, p_value = calculate_bartlett_sphericity(df_safety_test_s)
chi_square_value, p_value 

In [ ]:
#KMO검정
from factor_analyzer.factor_analyzer import calculate_kmo
kmo_all,kmo_model = calculate_kmo(df_safety_test_s)
kmo_model

In [ ]:
from factor_analyzer import FactorAnalyzer
fa = FactorAnalyzer(n_factors = 10, rotation = None)
fa.fit(df_safety_test_s)
ev,v = fa.get_eigenvalues()
ev

In [ ]:
plt.scatter(range(1,df_safety_test_s.shape[1]+1),ev)
plt.plot(range(1,df_safety_test_s.shape[1]+1),ev)
plt.title('Scree Plot')
plt.xlabel('Factors')
plt.ylabel('Eigenvalue')
plt.grid()
plt.show()

In [ ]:
fa = FactorAnalyzer(n_factors=3,rotation = "varimax")
fa.fit(df_safety_test_s)
efa_result = pd.DataFrame(fa.loadings_,index = df_safety_test_s.columns)
efa_result

In [ ]:
plt.figure(figsize = (4,6))
sns.heatmap(efa_result,cmap = "Blues",annot = True, fmt = '.2f')

In [ ]:
efa_result['mean'] = (efa_result[0] +efa_result[1] +efa_result[2])/3
efa_result

안전성_가중치 (총합1)

- 1위 발생건수 num: 0.25
- 2위 부상자수 injured :0.25
- 3위 10만명당 부상자수 injured_per100000:0.2
- 4위 차1만대당 발생건수 car10000_num:0.2
- 5위 사망자수 dead: 0.05
- 6위 10만명당 사망자수 dead_per100000:0.05
- 7위 교통안전지수: 0

# 2. 주거

In [ ]:
인구 = pd.read_csv('./주거/서울시 인구밀도 (동별) 통계.csv')

In [ ]:
생활인구 = pd.read_csv('./주거/행정동 단위 서울 생활인구(내국인).csv')